# Importing libs

In [33]:
import pandas as pd
import numpy as np
from snowflake.snowpark.session import Session
from snowflake.snowpark.functions import udf, avg, col
from snowflake.snowpark.types import IntegerType, FloatType, StringType, BooleanType
from snowflake.snowpark.files import SnowflakeFile
import sys
sys.path.append('..')
from credentials import Credentials

In [34]:
cred = Credentials()
session = Session.builder.configs(cred.__dict__).create()

In [35]:
session.use_role("SYSADMIN")
session.use_database("ACCOUNTADMIN_MGMT")
session.use_warehouse("ACCOUNTADMIN_MGMT")
session.use_schema("UTILITIES")

# Selecting from Mapping

## Individual table

In [ ]:
# Getting name of the table
db_name = 'STITCH'
schema_name = 'SALESFORCEFSL3'
table_name:str = 'GROUP'

In [ ]:
table_view_mapping = session.sql(f"SELECT * FROM ACCOUNTADMIN_MGMT.UTILITIES.SALESFORCE_TABLE_VIEW_MAPPING WHERE NAME_SALESFORCE_ENVIRONMENT = '{schema_name}' AND NAME_SALESFORCE_TABLE_ORIGINAL = '{table_name}'".format(schema_name, table_name)).collect()
table_view_mapping_dic = [row.asDict() for row in table_view_mapping][0]
table_name = table_view_mapping_dic['NAME_SALESFORCE_TABLE_ORIGINAL']
table_for_desc_name = table_view_mapping_dic['NAME_SNOWFLAKE_RESERVED_WORD']
view_name = table_view_mapping_dic['NAME_SALESFORCE_VIEW_ALIAS']

In [ ]:
result_describe_table = [row.as_dict() for row in session.sql(f"DESCRIBE TABLE {db_name}.{schema_name}.{table_for_desc_name};".format(db_name, schema_name, table_for_desc_name)).collect()]

In [ ]:
df_describe_table = pd.DataFrame(result_describe_table)
df_describe_table.drop(columns=['null?', 'default','primary key', 'unique key',
       'check', 'expression', 'comment', 'policy name'], inplace=True)

In [ ]:
df_describe_table.head(10)

## Stored Procedure sp_create_salesforce_views

In [36]:
def transform_name(name, type, name_alias):
    if isinstance(type, str) and 'VARCHAR' in type:
        #return name + '0007'
        return f"CAST(SUBSTR({name},1,10000) AS VARCHAR(10000)) AS {name_alias}".format(name, name_alias)
    else:
        return f"{name} AS {name_alias}".format(name, name_alias)

In [37]:
def sp_create_salesforce_views(session: Session, db_name: str, schema_name: str, table_name: str) -> str:
	# Getting name of the table
	table_view_mapping = session.sql(f"SELECT * FROM ACCOUNTADMIN_MGMT.UTILITIES.SALESFORCE_TABLE_VIEW_MAPPING WHERE NAME_SALESFORCE_ENVIRONMENT = '{schema_name}' AND NAME_SALESFORCE_TABLE_ORIGINAL = '{table_name}'".format(schema_name, table_name)).collect()
	table_view_mapping_dic = [row.asDict() for row in table_view_mapping][0]
	table_name = table_view_mapping_dic['NAME_SALESFORCE_TABLE_ORIGINAL']
	table_for_desc_name = table_view_mapping_dic['NAME_SNOWFLAKE_RESERVED_WORD']
	view_name = table_view_mapping_dic['NAME_SALESFORCE_VIEW_ALIAS']
	
	# Getting description of the table
	result_describe_table = [row.as_dict() for row in session.sql(f"DESCRIBE TABLE {db_name}.{schema_name}.{table_for_desc_name};".format(db_name, schema_name, table_for_desc_name)).collect()]

	# Removing unnecessary columns
	df_describe_table = pd.DataFrame(result_describe_table)
	df_describe_table.drop(columns=['null?', 'default','primary key', 'unique key',
       'check', 'expression', 'comment', 'policy name'], inplace=True)
	
	# Mixing with Mapping Table
	result_salesforce_mapping = [row.as_dict() for row in session.sql(f"SELECT NAME_SALESFORCE_ATTRIBUTE, NAME_ALIAS FROM ACCOUNTADMIN_MGMT.UTILITIES.SALESFORCE_MAPPING WHERE NAME_SALESFORCE_OBJECT = '{table_name}'".format()).collect()]
	df_salesforce_mapping = pd.DataFrame(result_salesforce_mapping)

	if df_salesforce_mapping.empty:
		final_df = df_describe_table
		final_df['NAME_ALIAS'] = final_df['name']
	else:
		final_df = pd.merge(df_describe_table, df_salesforce_mapping, left_on='name', right_on='NAME_SALESFORCE_ATTRIBUTE', how='left')
		final_df['NAME_ALIAS'] = final_df.apply(lambda x: x['name'] if pd.isna(x['NAME_ALIAS']) else x['NAME_ALIAS'], axis=1)

	# apply the transform_name function to create a new column called 'name_new'
	final_df['name_new'] = final_df.apply(lambda x: transform_name(x['name'], x['type'], x['NAME_ALIAS']), axis=1)

	column_list:list = final_df['name_new'].to_list()

	columns_str:str = ', '.join(column_list)

	# append the final result to something like "CREATE OR REPLACE VIEW {db_name}.{schema_name}}.{table_name}_V AS SELECT {columns_str} FROM {db_name}.{schema_name}.{table_name};"
	final_query:str = f"CREATE OR REPLACE VIEW {db_name}.{schema_name}.{view_name} AS SELECT {columns_str} FROM {db_name}.{schema_name}.{table_for_desc_name};"
	
	session.sql(final_query).collect()

	return "SUCCESS"

In [ ]:
# Getting name of the table
db_name = 'STITCH'
schema_name = 'SALESFORCEFSL3'
table_name:str = 'GROUP'

In [ ]:
sp_create_salesforce_views(session, db_name, schema_name, table_name)

## Register sp_create_salesforce_views

In [41]:
session.sproc.register(
	func=sp_create_salesforce_views,
	name="sp_create_salesforce_views",
	packages=["snowflake-snowpark-python", "pandas", "numpy"],
	is_permanent=True,
	stage_location="@ACCOUNTADMIN_MGMT.UTILITIES.ACCOUNTADMIN_MGMT_STAGE",
	source_code_display = False,
	execute_as = 'caller',
	replace=True
)

In [ ]:
session.call("sp_create_salesforce_views", db_name, schema_name, table_name)

# All Tables

In [42]:
def sp_run_dynamic_salesforce_views(session: Session) -> str:
    database:str = 'STITCH'

    for schema in ['SALESFORCEFSL3']:
        result = session.sql(f"SHOW TABLES IN STITCH.{schema}".format(schema)).collect()
        df = pd.DataFrame(result)
        tables:list = df['name'].to_list()
        for table in tables:
            if table not in ['_SDC_REJECTED']:
                print(table)
                session.call("sp_create_salesforce_views", database, schema, table)
    
    return "SUCCESS"

In [43]:
session.sproc.register(
	func=sp_run_dynamic_salesforce_views,
	name="sp_run_dynamic_salesforce_views",
	packages=["snowflake-snowpark-python", "pandas", "numpy"],
	is_permanent=True,
	stage_location="@ACCOUNTADMIN_MGMT.UTILITIES.ACCOUNTADMIN_MGMT_STAGE",
	source_code_display = False,
	execute_as = 'caller',
	replace=True
)

In [ ]:
session.call("sp_run_dynamic_salesforce_views")

## Creating Task

In [50]:
session.use_role("SYSADMIN")
session.use_database("ACCOUNTADMIN_MGMT")
session.use_warehouse("ACCOUNTADMIN_MGMT")
session.use_schema("UTILITIES")

In [51]:
task_run_dynamic_salesforce_views = """
CREATE OR REPLACE TASK ACCOUNTADMIN_MGMT.UTILITIES.TASK_RUN_DYNAMIC_SALESFORCE_VIEWS
    WAREHOUSE = 'ACCOUNTADMIN_MGMT'
    SCHEDULE = 'USING CRON 0 10 * * * CET'
    ALLOW_OVERLAPPING_EXECUTION = FALSE
    COMMENT = 'Run dynamic salesforce views'
AS
        CALL ACCOUNTADMIN_MGMT.UTILITIES.SP_RUN_DYNAMIC_SALESFORCE_VIEWS();
"""

In [52]:
session.sql(task_run_dynamic_salesforce_views).collect()

[Row(status='Task TASK_RUN_DYNAMIC_SALESFORCE_VIEWS successfully created.')]

In [53]:
session.sql("ALTER TASK ACCOUNTADMIN_MGMT.UTILITIES.TASK_RUN_DYNAMIC_SALESFORCE_VIEWS RESUME;").collect()

[Row(status='Statement executed successfully.')]

In [54]:
session.sql("EXECUTE TASK ACCOUNTADMIN_MGMT.UTILITIES.TASK_RUN_DYNAMIC_SALESFORCE_VIEWS;").collect()

[Row(status='Task TASK_RUN_DYNAMIC_SALESFORCE_VIEWS is scheduled to run immediately.')]

In [55]:
session.close()